Import Libraries

Note: This script has import features that will not work in Python 3

In [117]:
#imports
from __future__ import print_function
import pyart
import matplotlib.pyplot as plt
import matplotlib
#%matplotlib inline
import gc
import numpy as np
import os
from netCDF4 import num2date, date2num
from datetime import datetime
import h5py
import time
from mpl_toolkits.basemap import Basemap

Point to directory

In [114]:
dirr = '/home/nick-m/Code/uq-xpol-master/DataLinks/Odim/'
#dirr = '/home/nick-m/Desktop/AspeyTestData/'
outloc = '/home/nick-m/Desktop/AspeyReflOut/'
#outloc = '/home/nick-m/Desktop/AspeyTestOut/'
firename = 'Aspey'

#Read and sort
fls = os.listdir(dirr)
fls.sort()

flsPPI = [s for s in fls if "ppi" in s]
flsRHI = [s for s in fls if "rhi" in s]
print(str(len(flsPPI))+' PPIs')
print(str(len(flsRHI))+' RHIs')

2558 PPIs
315 RHIs


Plot Types? Set to 1 to plot

In [83]:
PlotRHIs = 0
PlotPPIs = 1

Setup Plotting Variables:

In [84]:
# SET Fields to be Plotted
fields_to_plot = ['DBZH', 'VRADH']

# SET No. of subplot rows
rows = 1 

tz = 10

# SET minimum reflectivity and field ranges

minref = -30
ranges = [(minref, 35), (-20, 20)]

# SET Colourmaps
colmaps = [pyart.graph.cm.NWSRef, 
            pyart.graph.cm.BuDRd18]


Setup Plot Type Specific Variables.

Note: Give plot area in lat/lon. Use the tool http://boundingbox.klokantech.com/ to output in correct format

In [85]:
## Plot type specifics

if PlotRHIs == 1:
    
    # SET Lims for RHIs
    RHIxlims, RHIylims = [5,20],[0, 7]


if PlotPPIs == 1:
    
    # SET PPI DOMAIN
    
    #                                     West      South   East    North 
    min_lon, min_lat, max_lon, max_lat = [150.96,-28.35,151.1,-28.2]
    
    # SET Linespacing on map
    lat_lin_space = 0.05
    lon_lin_space = 0.05
    
    # Make lines...
    lat_lines = np.arange(np.ceil(max_lat),np.floor(min_lat),-lat_lin_space)
    lon_lines = np.arange(np.floor(min_lon),np.ceil(max_lon),lon_lin_space)
    
    # Radar Location
    xpol_lat = -28.333759
    xpol_lon = 151.088839


    # Azimuth Correction
    azimuth_offset = 151.6 # Offset = 140 + where Declination = 10.6 
    
    # epsg code for GDA94 Projection
    GDA94 = 3112
    
    # ArcGIS Online Map Service to use (e.g. World_Topo_Map or ESRI_Imagery_World_2D)
    esriMap = 'World_Topo_Map'
    
    # Location of Shapefile to Overlay (TO BE FIXED)
    # overlayLoc = '/home/nick-m/Desktop/CedarMapParts/Export_OutputCedarPoints'
    


Import AWS data (to be cleaned up)

In [86]:
SW_CornerAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/SW_Corner.csv',
          dtype={'names': ("#","time","p","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
SW_CornerAWSlat = -28.290919
SW_CornerAWSlon = 151.010911

Water_nAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/Water_N.csv',
          dtype={'names': ("#","time","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
Water_nAWSlat = -28.235011
Water_nAWSlon = 151.012899

Water_sAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/Water_S.csv',
          dtype={'names': ("#","time","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
Water_sAWSlat = -28.2675
Water_sAWSlon = 151.035168

CowsClearingAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/CowsClearing.csv',
          dtype={'names': ("#","time","rad","T","RH","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
CowsClearingAWSlat = -28.301012
CowsClearingAWSlon = 151.064821

ClearingNthAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/Clearing_Nth.csv',
          dtype={'names': ("#","time","T","RH","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
ClearingNthAWSlat = -28.236314
ClearingNthAWSlon = 150.987566

SunsetStripAWS = np.loadtxt('/media/nick-m/Seagate Expansion Drive/BCPE_Field_Data/AspeyBurn/AspeyAWS/SunsetStrip.csv',
          dtype={'names': ("#","time","ws","gs","wd"),
                 'formats': ('i4',object,'f8','f8','f8','f8')},
          delimiter=',',skiprows=2, converters={1: lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")})
SunsetStripAWSlat = -28.261541
SunsetStripAWSlon = 150.994655

AWSlats = [SW_CornerAWSlat, Water_nAWSlat, Water_sAWSlat, CowsClearingAWSlat, ClearingNthAWSlat, SunsetStripAWSlat]
AWSlons = [SW_CornerAWSlon, Water_nAWSlon, Water_sAWSlon, CowsClearingAWSlon, ClearingNthAWSlon, SunsetStripAWSlon]

#Convert to km/h
SW_CornerAWS['ws'] = SW_CornerAWS['ws']*3.6
Water_nAWS['ws'] = Water_nAWS['ws']*3.6
Water_sAWS['ws'] = Water_sAWS['ws']*3.6
CowsClearingAWS['ws'] = CowsClearingAWS['ws']*3.6
ClearingNthAWS['ws'] = ClearingNthAWS['ws']*3.6
SunsetStripAWS['ws'] = SunsetStripAWS['ws']*3.6

In [131]:
triggertime = 0
#Due to the plotting loop slowing down, this variable tracks the length of the whole plot loop.
#Once the triggertime reaches 2.5 seconds, the 

In [87]:
def UVnearestDate(AWS, time):
    idx = np.argmin(np.abs(AWS['time'] - time))
    u = -AWS['ws'][idx] * np.sin(AWS['wd'][idx] * 0.01745329)  # 0.01745329 is pi/180
    v = -AWS['ws'][idx] * np.cos(AWS['wd'][idx] * 0.01745329)
    
    return u,v

Set the offset for the sppi

In [1]:
i = 1

if PlotPPIs == 1:
    start = time.time()
    #Setup Basemap
    m = Basemap(llcrnrlon=min_lon,
                llcrnrlat=min_lat,
                urcrnrlon=max_lon,
                urcrnrlat=max_lat, 
                projection='tmerc', 
                resolution = 'i',
                epsg = GDA94)
    
    AWSx, AWSy = m(AWSlons, AWSlats)

    print('Retrieving ArcGIS Online Image for Map Domain...')
    try:
        m.arcgisimage(service= esriMap, xpixels = 1500, verbose= True)
        print('Success')
    except:
            print('Image Retrieved Failed. Continuing with no image.')
    
        
    
    # Initiate loop. Basemap not replotted, so Colourbar only plotted on first iteration
    Mark1 = time.time()
    print('Map took ', str(round(Mark1 - start,2)), 'seconds')
    

    for fl in flsPPI:
#        try:

            print('doing ' + str(i) + ' of ' + str(len(flsPPI)) + ' PPIs - '+ dirr + fl)
            
            if triggertime > 2.5:
                plt.close()
                m = Basemap(llcrnrlon=min_lon,llcrnrlat=min_lat,urcrnrlon=max_lon,urcrnrlat=max_lat, projection='tmerc', resolution = 'i',epsg = GDA94)
                m.arcgisimage(service= esriMap, xpixels = 1500, verbose= True)
                print('\033[1m' + 'Fig closed, new basemap initialised' + '\033[0m')
        
        
            Mark2 = time.time()
            
            
            #Load Data
            myradar = pyart.aux_io.read_odim_h5(dirr + fl, file_field_names=True)
            
            
            
            
            #Extract data (timezone *60*60 as datenum is given in seconds since...)
            dts = num2date(myradar.time['data'] + tz* 60.*60., myradar.time['units'])
            
            
            

            # Set gatefilters
            gatefilter = pyart.correct.GateFilter(myradar)
            gatefilter.exclude_inside('VRADH',-0.2,0.2)
            gatefilter.exclude_outside('VRADH',-12,12)
            gatefilter = pyart.correct.despeckle.despeckle_field(myradar,
                                                             'VRADH', 
                                                             gatefilter=gatefilter, 
                                                             size = 100)
            
            
        
            #rename fields
            myradar.fields['DBZH']['standard_name'] = 'Reflectivity'
            myradar.fields['DBZH']['units'] = 'dBZ'
            myradar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
            myradar.fields['VRADH']['standard_name'] = 'Velocity'
            myradar.fields['VRADH']['units'] = 'm/s'
            myradar.fields['VRADH']['long_name'] = 'Radial Velocity of Scatterers'
            
            
            
        
            # Geographical Corrections
            myradar.latitude['data'][0] = xpol_lat 
            myradar.longitude['data'][0] = xpol_lon
            myradar.azimuth['data'] = myradar.azimuth['data'] + azimuth_offset
            
            
            
            
            #Mark3 = time.time()
            #print('Load and corrections took ', str(round(Mark3 - Mark2,2)), 'seconds')
            
            
            
            #Setup Display
            my_display = pyart.graph.RadarMapDisplay(myradar)
            
            
            
            #Mark4 = time.time()
            #print('Setup Display took ', str(round(Mark4 - Mark3,2)), 'seconds')
            
            

            # Shapefile (TO BE FIXED)
            #m.readshapefile(overlayLoc, 'FirePerim')
            
        
        
            # Only plot colorbar on the first plot
        
            if i == 1:
                cbf = True
            else:
                cbf = False
                
                
            
            #Plot Radar Data

            my_display.plot_ppi_map('DBZH', vmin=-15, vmax=35, 
                                    lat_lines = lat_lines, lon_lines = lon_lines,
                                    #max_lat = max_lat, #NorthLatitude,
                                    #min_lat = min_lat, #SouthLatitude,
                                    #min_lon = min_lon, #WestLongitude, 
                                    #max_lon = max_lon, #EastLongitude,
                                    cmap = pyart.graph.cm.NWSRef,
                                    gatefilter=gatefilter,
                                    colorbar_flag = cbf,
                                    basemap = m,
                                   )

            #my_display.plot_range_ring(2.)
            #my_display.plot_range_ring(4.)
            #my_display.plot_range_ring(6.)
            
            Mark5 = time.time()
            print('Plot took ', str(round(Mark5 - Mark2,2)), 'seconds')
            
            
            my_display.plot_range_ring(10.)
            my_display.plot_point(xpol_lon, xpol_lat, symbol = 'wo')


           
        
            # Plot windbarbs
            
            uSW_CornerAWS,vSW_CornerAWS = UVnearestDate(SW_CornerAWS,dts[0])
            uWater_nAWS,vWater_nAWS = UVnearestDate(Water_nAWS,dts[0])
            uWater_sAWS,vWater_sAWS = UVnearestDate(Water_sAWS,dts[0])
            uCowsClearingAWS,vCowsClearingAWS = UVnearestDate(CowsClearingAWS,dts[0])
            uClearingNthAWS,vClearingNthAWS = UVnearestDate(ClearingNthAWS,dts[0])
            uSunsetStripAWS,vSunsetStripAWS = UVnearestDate(SunsetStripAWS,dts[0])
            
            uData = [uSW_CornerAWS, uWater_nAWS, uWater_sAWS, uCowsClearingAWS, uClearingNthAWS, uSunsetStripAWS]
            vData = [vSW_CornerAWS, vWater_nAWS, vWater_sAWS, vCowsClearingAWS, vClearingNthAWS, vSunsetStripAWS]
            
            #plt.quiver(AWSx, AWSy,uData,vData,scale=700)
            plt.barbs(AWSx, AWSy,uData,vData,length=8, pivot='middle')
            
            Mark6 = time.time()
            #print('Barbs took ', str(round(Mark6 - Mark5,2)), 'seconds')
            
            
            
            
            #Create title/name and save fig
            
            t = ('PPI over ' + firename +' at ' + 
                 dts[0].strftime('%H:%M:%S %Y-%m-%d'))
            plt.title(t, fontweight='bold')
            
            #datest = dts[0]

        
            fn = (firename +
                  fl[7:-18] +
                  'el_' + str(int(myradar.elevation['data'][0])) +
                  '_' + fl[-18:-3])
            
            #fn = (firename + '_' + myradar.scan_type + str(i)) 

            plt.savefig(outloc + fn + '.png', dpi=300)
            
            Mark7 = time.time()
            print('Save Plot took ', str(round(Mark7 - Mark6,2)), 'seconds')
            
            ax = plt.gca()
            
            
            
            
            ## Below operations seek to clear out unused layers in the plot to avoid 
            
            #plt.savefig(outloc + 'test' + fn + '.png', dpi=400) 
            #print(len(ax.collections))
            
            ax.collections.pop(-4)
            ax.collections.pop(-3)
            ax.collections.pop(-2)
            ax.collections.pop(-1)
            
            print(len(ax.collections))
            #print(ax.get_zorder)
            #print(ax(1))
            #print(ax.axes)

            #fig.delaxes(1)
            
            #fig.set_ydata()
            #fig.set_xdata()
            #self.figure.canvas.draw()
            
            #print(len(locals()))
            
            #fig.canvas.update()
            #fig.canvas.flush_events()
            
            #plt.savefig(outloc + 'test2' + fn + '.png', dpi=200)
            
            Mark8 = time.time()
            print('\033[1m' + 'Total took ', str(round(Mark8 - Mark2,2)), 'seconds' + '\033[0m')
            
            triggertime = round(Mark8 - Mark2,2)
            
            i = i + 1
        
            #if i == 5:
            #    plt.close()
            #    break
#        except:
#            pass

    plt.close()

NameError: name 'plt' is not defined

In [127]:
#plt.close()

(0.29259959149818243, -0.47747824982411169)

(0.91914660149617511, 0.62230983035625553)

Hello
['Map took ', 0.00029397010803222656]
